In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7030,2021-01-23T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1444,223,1667,45960,...,NaN,14.0,NaN,NaN,128877.0,0.0,1431974.0,151914.0,ITG,ITG2
7031,2021-01-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,629,107,736,7525,...,NaN,8.0,NaN,NaN,130295.0,38.0,2078051.0,45882.0,ITG,ITG1
7032,2021-01-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,292,46,338,4691,...,NaN,5.0,NaN,NaN,33822.0,0.0,567260.0,11238.0,ITI,ITI2
7033,2021-01-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,43,2,45,282,...,NaN,0.0,NaN,NaN,7720.0,0.0,67704.0,444.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7030,2021-01-23T17:00:00,19,Sicilia,1444,223,1667,45960,47627,338,1158,...,NaN,NaN,128877,1583888,940932.0,14.0,128877.0,0.0,1431974.0,151914.0
7031,2021-01-23T17:00:00,9,Toscana,629,107,736,7525,8261,59,520,...,NaN,NaN,130333,2123933,1145629.0,8.0,130295.0,38.0,2078051.0,45882.0
7032,2021-01-23T17:00:00,10,Umbria,292,46,338,4691,5029,149,332,...,NaN,NaN,33822,578498,273719.0,5.0,33822.0,0.0,567260.0,11238.0
7033,2021-01-23T17:00:00,2,Valle d'Aosta,43,2,45,282,327,-4,5,...,NaN,NaN,7720,68148,40839.0,0.0,7720.0,0.0,67704.0,444.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-23', '2021-01-22')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-23T17:00:00,1,Piemonte,2337,156,2493,10962,13455,99,931,...,NaN,NaN,217058,1931124,1061000.0,19.0,216047.0,1011.0,1827915.0,103209.0
1,2021-01-23T17:00:00,2,Valle d'Aosta,43,2,45,282,327,-4,5,...,NaN,NaN,7720,68148,40839.0,0.0,7720.0,0.0,67704.0,444.0
2,2021-01-23T17:00:00,3,Lombardia,3499,398,3897,48571,52468,-747,1535,...,NaN,NaN,523899,5414816,2773895.0,14.0,523621.0,278.0,5346945.0,67871.0
3,2021-01-23T17:00:00,5,Veneto,1843,286,2129,45412,47541,-2309,1030,...,NaN,NaN,304717,3880349,1301596.0,18.0,304717.0,0.0,3657997.0,222352.0
4,2021-01-23T17:00:00,6,Friuli Venezia Giulia,672,63,735,11163,11898,-192,679,...,NaN,NaN,63833,1087567,397725.0,5.0,62423.0,1410.0,1067967.0,19600.0
5,2021-01-23T17:00:00,7,Liguria,591,65,656,3914,4570,52,298,...,NaN,NaN,67400,812181,358768.0,9.0,67400.0,0.0,791490.0,20691.0
6,2021-01-23T17:00:00,8,Emilia-Romagna,2390,217,2607,48125,50732,-479,1310,...,NaN,NaN,209235,2932963,1322682.0,13.0,209213.0,22.0,2879137.0,53826.0
7,2021-01-23T17:00:00,9,Toscana,629,107,736,7525,8261,59,520,...,NaN,NaN,130333,2123933,1145629.0,8.0,130295.0,38.0,2078051.0,45882.0
8,2021-01-23T17:00:00,10,Umbria,292,46,338,4691,5029,149,332,...,NaN,NaN,33822,578498,273719.0,5.0,33822.0,0.0,567260.0,11238.0
9,2021-01-23T17:00:00,11,Marche,549,77,626,7785,8411,140,397,...,NaN,NaN,52378,662555,392606.0,3.0,52378.0,0.0,647536.0,15019.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-22T17:00:00,1,Piemonte,2342,152,2494,10862,13356,-237,849,...,NaN,NaN,216127,1907355,1055264.0,0.0,215230.0,897.0,1819262.0,88093.0
1,2021-01-22T17:00:00,2,Valle d'Aosta,40,2,42,289,331,-10,19,...,NaN,NaN,7715,67890,40791.0,0.0,7715.0,0.0,67539.0,351.0
2,2021-01-22T17:00:00,3,Lombardia,3579,408,3987,49228,53215,160,1969,...,NaN,NaN,522364,5383007,2766601.0,21.0,522095.0,269.0,5323433.0,59574.0
3,2021-01-22T17:00:00,5,Veneto,1935,291,2226,47624,49850,-3707,1198,...,NaN,NaN,303687,3830872,1298664.0,15.0,303687.0,0.0,3644507.0,186365.0
4,2021-01-22T17:00:00,6,Friuli Venezia Giulia,669,61,730,11360,12090,-90,452,...,NaN,NaN,63154,1077328,393779.0,6.0,61983.0,1171.0,1060923.0,16405.0
5,2021-01-22T17:00:00,7,Liguria,615,61,676,3842,4518,-85,313,...,NaN,NaN,67102,805701,356370.0,3.0,67102.0,0.0,787636.0,18065.0
6,2021-01-22T17:00:00,8,Emilia-Romagna,2402,220,2622,48589,51211,-826,1347,...,NaN,NaN,207925,2908287,1318796.0,15.0,207914.0,11.0,2865735.0,42552.0
7,2021-01-22T17:00:00,9,Toscana,642,110,752,7450,8202,-101,429,...,NaN,NaN,129813,2108893,1139530.0,6.0,129781.0,32.0,2068043.0,40850.0
8,2021-01-22T17:00:00,10,Umbria,283,44,327,4553,4880,159,305,...,NaN,NaN,33490,572082,272550.0,4.0,33490.0,0.0,563514.0,8568.0
9,2021-01-22T17:00:00,11,Marche,555,79,634,7637,8271,197,437,...,NaN,NaN,51981,653333,388208.0,1.0,51981.0,0.0,641147.0,12186.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-23T17:00:00,1,Piemonte,2337,156,2493,10962,13455,99,931,...,NaN,NaN,217058,1931124,1061000.0,19.0,216047.0,1011.0,1827915.0,103209.0
1,2021-01-23T17:00:00,2,Valle d'Aosta,43,2,45,282,327,-4,5,...,NaN,NaN,7720,68148,40839.0,0.0,7720.0,0.0,67704.0,444.0
2,2021-01-23T17:00:00,3,Lombardia,3499,398,3897,48571,52468,-747,1535,...,NaN,NaN,523899,5414816,2773895.0,14.0,523621.0,278.0,5346945.0,67871.0
3,2021-01-23T17:00:00,4,Trentino-Alto Adige,422,67,489,14944,15433,347,745,...,0.0,0.0,62275,950938,333937.0,4.0,59174.0,3101.0,903202.0,47736.0
4,2021-01-23T17:00:00,5,Veneto,1843,286,2129,45412,47541,-2309,1030,...,NaN,NaN,304717,3880349,1301596.0,18.0,304717.0,0.0,3657997.0,222352.0
5,2021-01-23T17:00:00,6,Friuli Venezia Giulia,672,63,735,11163,11898,-192,679,...,NaN,NaN,63833,1087567,397725.0,5.0,62423.0,1410.0,1067967.0,19600.0
6,2021-01-23T17:00:00,7,Liguria,591,65,656,3914,4570,52,298,...,NaN,NaN,67400,812181,358768.0,9.0,67400.0,0.0,791490.0,20691.0
7,2021-01-23T17:00:00,8,Emilia-Romagna,2390,217,2607,48125,50732,-479,1310,...,NaN,NaN,209235,2932963,1322682.0,13.0,209213.0,22.0,2879137.0,53826.0
8,2021-01-23T17:00:00,9,Toscana,629,107,736,7525,8261,59,520,...,NaN,NaN,130333,2123933,1145629.0,8.0,130295.0,38.0,2078051.0,45882.0
9,2021-01-23T17:00:00,10,Umbria,292,46,338,4691,5029,149,332,...,NaN,NaN,33822,578498,273719.0,5.0,33822.0,0.0,567260.0,11238.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-01-22T17:00:00,1,Piemonte,2342,152,2494,10862,13356,-237,...,NaN,NaN,216127,1907355,1055264.0,0.0,215230.0,897.0,1819262.0,88093.0
1,1,2021-01-22T17:00:00,2,Valle d'Aosta,40,2,42,289,331,-10,...,NaN,NaN,7715,67890,40791.0,0.0,7715.0,0.0,67539.0,351.0
2,2,2021-01-22T17:00:00,3,Lombardia,3579,408,3987,49228,53215,160,...,NaN,NaN,522364,5383007,2766601.0,21.0,522095.0,269.0,5323433.0,59574.0
3,19,2021-01-22T17:00:00,4,Trentino-Alto Adige,446,65,511,14575,15086,257,...,0.0,0.0,61530,938927,332880.0,1.0,58868.0,2662.0,898168.0,40759.0
4,3,2021-01-22T17:00:00,5,Veneto,1935,291,2226,47624,49850,-3707,...,NaN,NaN,303687,3830872,1298664.0,15.0,303687.0,0.0,3644507.0,186365.0
5,4,2021-01-22T17:00:00,6,Friuli Venezia Giulia,669,61,730,11360,12090,-90,...,NaN,NaN,63154,1077328,393779.0,6.0,61983.0,1171.0,1060923.0,16405.0
6,5,2021-01-22T17:00:00,7,Liguria,615,61,676,3842,4518,-85,...,NaN,NaN,67102,805701,356370.0,3.0,67102.0,0.0,787636.0,18065.0
7,6,2021-01-22T17:00:00,8,Emilia-Romagna,2402,220,2622,48589,51211,-826,...,NaN,NaN,207925,2908287,1318796.0,15.0,207914.0,11.0,2865735.0,42552.0
8,7,2021-01-22T17:00:00,9,Toscana,642,110,752,7450,8202,-101,...,NaN,NaN,129813,2108893,1139530.0,6.0,129781.0,32.0,2068043.0,40850.0
9,8,2021-01-22T17:00:00,10,Umbria,283,44,327,4553,4880,159,...,NaN,NaN,33490,572082,272550.0,4.0,33490.0,0.0,563514.0,8568.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-01-23T17:00:00,1,Piemonte,2337,156,2493,10962,13455,99,931,...,NaN,217058,1931124,1061000.0,19.0,216047.0,1011.0,1827915.0,103209.0,4341375
1,2021-01-23T17:00:00,2,Valle d'Aosta,43,2,45,282,327,-4,5,...,NaN,7720,68148,40839.0,0.0,7720.0,0.0,67704.0,444.0,125501
2,2021-01-23T17:00:00,3,Lombardia,3499,398,3897,48571,52468,-747,1535,...,NaN,523899,5414816,2773895.0,14.0,523621.0,278.0,5346945.0,67871.0,10103969
3,2021-01-23T17:00:00,4,Trentino-Alto Adige,422,67,489,14944,15433,347,745,...,0.0,62275,950938,333937.0,4.0,59174.0,3101.0,903202.0,47736.0,1074819
4,2021-01-23T17:00:00,5,Veneto,1843,286,2129,45412,47541,-2309,1030,...,NaN,304717,3880349,1301596.0,18.0,304717.0,0.0,3657997.0,222352.0,4907704
5,2021-01-23T17:00:00,6,Friuli Venezia Giulia,672,63,735,11163,11898,-192,679,...,NaN,63833,1087567,397725.0,5.0,62423.0,1410.0,1067967.0,19600.0,1211357
6,2021-01-23T17:00:00,7,Liguria,591,65,656,3914,4570,52,298,...,NaN,67400,812181,358768.0,9.0,67400.0,0.0,791490.0,20691.0,1543127
7,2021-01-23T17:00:00,8,Emilia-Romagna,2390,217,2607,48125,50732,-479,1310,...,NaN,209235,2932963,1322682.0,13.0,209213.0,22.0,2879137.0,53826.0,4467118
8,2021-01-23T17:00:00,9,Toscana,629,107,736,7525,8261,59,520,...,NaN,130333,2123933,1145629.0,8.0,130295.0,38.0,2078051.0,45882.0,3722729
9,2021-01-23T17:00:00,10,Umbria,292,46,338,4691,5029,149,332,...,NaN,33822,578498,273719.0,5.0,33822.0,0.0,567260.0,11238.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-01-23T17:00:00,1,Piemonte,2337,156,2493,10962,13455,99,931,...,217058,1931124,1061000.0,19.0,216047.0,1011.0,1827915.0,103209.0,4341375,575
1,2021-01-23T17:00:00,2,Valle d'Aosta,43,2,45,282,327,-4,5,...,7720,68148,40839.0,0.0,7720.0,0.0,67704.0,444.0,125501,20
2,2021-01-23T17:00:00,3,Lombardia,3499,398,3897,48571,52468,-747,1535,...,523899,5414816,2773895.0,14.0,523621.0,278.0,5346945.0,67871.0,10103969,1036
3,2021-01-23T17:00:00,4,Trentino-Alto Adige,422,67,489,14944,15433,347,745,...,62275,950938,333937.0,4.0,59174.0,3101.0,903202.0,47736.0,1074819,106
4,2021-01-23T17:00:00,5,Veneto,1843,286,2129,45412,47541,-2309,1030,...,304717,3880349,1301596.0,18.0,304717.0,0.0,3657997.0,222352.0,4907704,1016
5,2021-01-23T17:00:00,6,Friuli Venezia Giulia,672,63,735,11163,11898,-192,679,...,63833,1087567,397725.0,5.0,62423.0,1410.0,1067967.0,19600.0,1211357,180
6,2021-01-23T17:00:00,7,Liguria,591,65,656,3914,4570,52,298,...,67400,812181,358768.0,9.0,67400.0,0.0,791490.0,20691.0,1543127,209
7,2021-01-23T17:00:00,8,Emilia-Romagna,2390,217,2607,48125,50732,-479,1310,...,209235,2932963,1322682.0,13.0,209213.0,22.0,2879137.0,53826.0,4467118,634
8,2021-01-23T17:00:00,9,Toscana,629,107,736,7525,8261,59,520,...,130333,2123933,1145629.0,8.0,130295.0,38.0,2078051.0,45882.0,3722729,523
9,2021-01-23T17:00:00,10,Umbria,292,46,338,4691,5029,149,332,...,33822,578498,273719.0,5.0,33822.0,0.0,567260.0,11238.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-23T17:00:00,3,Lombardia,3499,398,3897,48571,52468,-747,1535,...,38.42,-657,-434,31809,7294.0,104,2178,4.83,0.01519,5.19
1,2021-01-23T17:00:00,8,Emilia-Romagna,2390,217,2607,48125,50732,-479,1310,...,34.23,-464,-37,24676,3886.0,33,1756,5.31,0.02933,4.68
2,2021-01-23T17:00:00,12,Lazio,2576,283,2859,67407,70266,-1180,1297,...,33.41,-1077,156,30588,16240.0,42,2435,4.24,0.02211,3.35
3,2021-01-23T17:00:00,19,Sicilia,1444,223,1667,45960,47627,338,1158,...,37.93,334,-197,23465,5721.0,33,787,4.94,0.02331,2.59
4,2021-01-23T17:00:00,15,Campania,1437,114,1551,60902,62453,320,1150,...,22.57,311,44,15663,11457.0,29,801,7.34,0.01988,3.68
5,2021-01-23T17:00:00,5,Veneto,1843,286,2129,45412,47541,-2309,1030,...,28.15,-2212,-168,49477,2932.0,92,3247,2.08,0.02099,6.21
6,2021-01-23T17:00:00,16,Puglia,1381,159,1540,53075,54615,68,1023,...,43.09,8,5,10333,3601.0,11,944,9.90,0.02552,2.87
7,2021-01-23T17:00:00,1,Piemonte,2337,156,2493,10962,13455,99,931,...,27.13,100,82,23769,5736.0,26,806,3.92,0.02144,5.00
8,2021-01-23T17:00:00,4,Trentino-Alto Adige,422,67,489,14944,15433,347,745,...,63.21,369,56,12011,1057.0,9,389,6.20,0.06931,5.79
9,2021-01-23T17:00:00,6,Friuli Venezia Giulia,672,63,735,11163,11898,-192,679,...,35.00,-197,227,10239,3946.0,24,847,6.63,0.05605,5.27


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,67,39,106,63.21
1,Marche,77,66,143,53.85
2,Puglia,159,210,369,43.09
3,Umbria,46,71,117,39.32
4,Lombardia,398,638,1036,38.42
5,Sicilia,223,365,588,37.93
6,Friuli Venezia Giulia,63,117,180,35.00
7,Emilia-Romagna,217,417,634,34.23
8,Lazio,283,564,847,33.41
9,Liguria,65,144,209,31.10


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,2021-01-21T17:00:00,22,P.A. Trento,236,43,279,1727,2006,-80,164,...,NaN,NaN,25714,498378,158775.0,3.0,24948.0,766.0,491561.0,6817.0
666,2021-01-22T17:00:00,21,P.A. Bolzano,227,24,251,12790,13041,218,510,...,NaN,NaN,35637,436669,173733.0,0.0,33861.0,1776.0,403835.0,32834.0
667,2021-01-22T17:00:00,22,P.A. Trento,219,41,260,1785,2045,39,179,...,NaN,NaN,25893,502258,159147.0,1.0,25007.0,886.0,494333.0,7925.0
668,2021-01-23T17:00:00,21,P.A. Bolzano,219,24,243,13143,13386,345,552,...,NaN,NaN,36189,444641,174483.0,1.0,34108.0,2081.0,406253.0,38388.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2021-01-19T17:00:00,4,Trentino-Alto Adige,483,76,559,13995,14554,78,579,...,14.0,10436.0,954.0,96.0,13.60,3.84,5.55,60.69,0.00,0.54
331,2021-01-20T17:00:00,4,Trentino-Alto Adige,473,72,545,14073,14618,64,719,...,8.0,11804.0,939.0,78.0,13.21,3.73,6.09,76.57,-5.26,0.44
332,2021-01-21T17:00:00,4,Trentino-Alto Adige,462,71,533,14296,14829,211,730,...,14.0,11631.0,1196.0,223.0,13.32,3.59,6.28,61.04,-1.39,1.44
333,2021-01-22T17:00:00,4,Trentino-Alto Adige,446,65,511,14575,15086,257,689,...,4.0,10716.0,892.0,279.0,12.72,3.39,6.43,77.24,-8.45,1.73


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
